In [ ]:
import sys
import os

sys.path.append("../../../mmlab_utils/")

In [ ]:
import logging

logging.basicConfig(level=logging.ERROR)

In [ ]:
from mmengine.registry import init_default_scope

init_default_scope("mmdet")

In [ ]:
from collections import defaultdict
import torch
from shapely.geometry import Polygon
from tqdm import tqdm
from mmlab_utils.mmdet_utils import Coco2Dataset
from mmlab_utils.mmdet_utils.apis.base_loader import BasePredictor
from mmlab_utils.mmdet_utils.apis.base_loader import find_last_model

In [ ]:
model_folder = "/data/projects/yolov8_v1/YOLOV8_v1_scale_640/"

config_file, checkpoint = find_last_model(model_folder)

print(f"{config_file=}")
print(f"{checkpoint=}")

In [ ]:
detector = BasePredictor(model_folder, min_score=0.7, use_cpu=False)
detector

In [ ]:
pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="mmdet.LoadAnnotations", with_bbox=True),
    dict(
        type="mmdet.PackDetInputs",
        meta_keys=(
            "img",
            "img_id",
            "img_path",
            "ori_shape",
            "img_shape",
            "instances",
        ),
    ),
]

dataset = Coco2Dataset(
    "../det_stones/dataset/test/",
    classes=detector.inference_model.inference_model.dataset_meta["classes"],
    pipeline=pipeline,
    # MAX_SIZE=detector.MAX_SIZE[0]
)

len(dataset)

In [ ]:
data_samples = []

for i, item in tqdm(enumerate(dataset)):
    data_sample = item["data_samples"]

    img = data_sample.img

    _result_v3 = detector.inference_model.inference_detector([img])[0]
    data_sample.pred_instances = _result_v3.pred_instances

    data_samples.append(data_sample.to_dict())

In [ ]:
from mmlab_utils.mmdet_utils.evaluation import FasterCocoMetric
import numpy as np

iou_thrs = np.linspace(
    0.2, 0.95, int(np.round((0.95 - 0.5) / 0.05)) + 1, endpoint=True
).round(2)

metric = FasterCocoMetric(
    metric="segm" if detector.mask_model else "bbox",
    classwise=True,
    iou_thrs=iou_thrs,
)

In [ ]:
metric.dataset_meta = detector.inference_model.inference_model.dataset_meta

In [ ]:
metric.process({}, data_samples)

In [ ]:
metrics_dict = metric.evaluate(len(data_samples))